In [3]:
# Accedemos a nuestras claves API con dotenv
import os
from dotenv import load_dotenv
from langchain_openai import OpenAI, ChatOpenAI
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pandas as pd

# Cargar las variables de entorno
load_dotenv()
# OpenAI API Key
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
# Definir la clase para el sistema RAG
class SistemaRAG:
    def __init__(self, ruta_archivo, api_key, chunk_size=1000, chunk_overlap=0):
        """
        Inicializa el sistema RAG con los parámetros proporcionados.
        :param ruta_archivo: Ruta al archivo que contiene los datos de entrada.
        :param api_key: Clave API para acceder a los servicios de OpenAI.
        :param chunk_size: Tamaño de cada chunk al dividir el documento.
        :param chunk_overlap: Solapamiento entre los chunks de texto.
        """
        self.ruta_archivo = ruta_archivo
        self.api_key = api_key
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.vector_db = None
        self.retriever = None
        self.chat_model = ChatOpenAI(model="gpt-4o-mini", api_key=self.api_key)

        # Inicializar la carga de documentos y procesamiento
        self._cargar_y_procesar_documento()

    def _cargar_y_procesar_documento(self):
        """
        Carga el documento y lo procesa para dividirlo en chunks y crear la base de datos vectorial.
        """
        # Cargar documento
        loader = TextLoader(self.ruta_archivo)
        loaded_document = loader.load()

        # Dividir el documento en chunks
        text_splitter = CharacterTextSplitter(chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
        chunks_of_text = text_splitter.split_documents(loaded_document)

        # Crear los embeddings usando OpenAI
        embeddings = OpenAIEmbeddings(api_key=self.api_key)

        # Crear la base de datos vectorial con FAISS
        self.vector_db = FAISS.from_documents(chunks_of_text, embeddings)

        # Crear el retriever para buscar documentos
        self.retriever = self.vector_db.as_retriever(search_kwargs={"k": 5})

    def realizar_consulta(self, consulta):
        """
        Realiza una consulta al sistema RAG y devuelve la respuesta.
        :param consulta: La consulta a realizar.
        :return: Respuesta generada por el sistema RAG.
        """
        # Crear el template de prompt
        prompt_template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
        prompt = ChatPromptTemplate.from_template(prompt_template)

        # Crear una función para formatear los documentos recuperados
        def format_docs(docs):
            return "\n\n".join([d.page_content for d in docs])

        # Crear la cadena completa para invocar al modelo
        chain = (
                {"context": self.retriever | format_docs, "question": RunnablePassthrough()}
                | prompt
                | self.chat_model
                | StrOutputParser()
        )

        # Realizar la consulta
        response = chain.invoke(consulta)
        return response

In [6]:
# Ejemplo de uso del sistema RAG
ruta_csv = "../data/raw_data/accessories.csv"
sistema_rag = SistemaRAG(ruta_archivo=ruta_csv, api_key=openai_api_key)

# Realizar una consulta al sistema RAG
consulta = "What is the price of the black 3D glasses?"
response = sistema_rag.realizar_consulta(consulta)
print(response)

The price of the black 3D glasses is 490.


In [7]:
# Definir la clase para cargar datos CSV
class CargadorDatosCSV:
    def __init__(self, ruta_archivo):
        """
        Inicializa la clase con la ruta del archivo CSV.
        :param ruta_archivo: Ruta al archivo CSV que contiene los datos de inventario.
        """
        self.ruta_archivo = ruta_archivo
        self.df = None

    def cargar_datos(self):
        """
        Carga los datos desde el archivo CSV especificado y los almacena en un DataFrame.
        """
        try:
            self.df = pd.read_csv(self.ruta_archivo)
            # Añadir una columna de 'Stock' si no existe en el CSV
            if 'Stock' not in self.df.columns:
                self.df['Stock'] = 0  # Inicializar todos los productos con stock 0
            print("Datos cargados exitosamente.")
            # Mostrar las columnas cargadas
            print("Columnas del DataFrame:", self.df.columns.tolist())
        except FileNotFoundError:
            print(f"Error: El archivo en la ruta '{self.ruta_archivo}' no fue encontrado.")
        except pd.errors.EmptyDataError:
            print("Error: El archivo CSV está vacío.")
        except Exception as e:
            print(f"Error inesperado: {e}")

In [8]:
# Crear la instancia del cargador de datos CSV
cargador = CargadorDatosCSV(ruta_csv)
cargador.cargar_datos()

Datos cargados exitosamente.
Columnas del DataFrame: ['Name', 'Variation', 'DIY', 'Buy', 'Sell', 'Color 1', 'Color 2', 'Size', 'Miles Price', 'Source', 'Source Notes', 'Seasonal Availability', 'Mannequin Piece', 'Version', 'Style', 'Label Themes', 'Type', 'Villager Equippable', 'Catalog', 'Filename', 'Internal ID', 'Unique Entry ID', 'Stock']


In [9]:
# Definir la clase para gestionar el stock
class GestorStock:
    def __init__(self, df):
        """
        Inicializa la clase con el DataFrame del inventario.
        :param df: DataFrame que contiene los datos del inventario.
        """
        self.df = df

    def actualizar_stock(self, producto_id, cantidad):
        """
        Actualiza el stock de un producto.
        :param producto_id: ID del producto a actualizar.
        :param cantidad: Nueva cantidad a asignar.
        """
        # Verificar que la columna exista y el producto también
        id_col = 'Unique Entry ID'  # Actualizamos el nombre de la columna de acuerdo con el CSV
        cantidad_col = 'Stock'  # Nombre que asignamos a la columna de cantidad de stock

        if id_col in self.df.columns:
            if producto_id in self.df[id_col].values:
                self.df.loc[self.df[id_col] == producto_id, cantidad_col] = cantidad
                print(f"Stock actualizado para el producto con ID {producto_id}. Nueva cantidad: {cantidad}")
            else:
                print(f"Producto con ID {producto_id} no encontrado.")
        else:
            print(f"No se encontró la columna '{id_col}' en el DataFrame.")

    def verificar_stock(self, producto_id):
        """
        Verifica la cantidad de stock de un producto específico.
        :param producto_id: ID del producto a verificar.
        :return: Cantidad de stock disponible.
        """
        id_col = 'Unique Entry ID'
        cantidad_col = 'Stock'

        if id_col in self.df.columns:
            if producto_id in self.df[id_col].values:
                stock = self.df.loc[self.df[id_col] == producto_id, cantidad_col].values[0]
                print(f"Cantidad de stock para el producto con ID {producto_id}: {stock}")
                return stock
            else:
                print(f"Producto con ID {producto_id} no encontrado.")
        else:
            print(f"No se encontró la columna '{id_col}' en el DataFrame.")
        return None

    def obtener_dataframe(self):
        """
        Retorna el DataFrame actualizado.
        :return: DataFrame actualizado.
        """
        return self.df

In [10]:
# Inicializamos el gestor del stock con los datos cargados
gestor_stock = GestorStock(cargador.df)

In [11]:
# Actualizamos el stock de un producto específico
gestor_stock.actualizar_stock(producto_id='FNxEraBTeWRiCvtFu', cantidad=45)

Stock actualizado para el producto con ID FNxEraBTeWRiCvtFu. Nueva cantidad: 45


In [12]:
# Verificar la cantidad de stock del producto actualizado
gestor_stock.verificar_stock(producto_id='FNxEraBTeWRiCvtFu')

Cantidad de stock para el producto con ID FNxEraBTeWRiCvtFu: 45


45

In [13]:
# Verificar el DataFrame actualizado
df_actualizado = gestor_stock.obtener_dataframe()
df_actualizado.head()

,Name,Variation,DIY,Buy,Sell,Color 1,Color 2,Size,Miles Price,Source,...,Version,Style,Label Themes,Type,Villager Equippable,Catalog,Filename,Internal ID,Unique Entry ID,Stock
0,3D glasses,White,No,490,122,White,Colorful,1x1,NaN,Able Sisters,...,1.0.0,Active,party,AccessoryEye,Yes,For sale,AccessoryGlassThreed0,4463,FNxEraBTeWRiCvtFu,45
1,3D glasses,Black,No,490,122,Black,Colorful,1x1,NaN,Able Sisters,...,1.0.0,Active,party,AccessoryEye,Yes,For sale,AccessoryGlassThreed1,11020,mM9SXPCcGPfPJAmtm,0
2,bandage,Beige,No,140,35,Beige,White,1x1,NaN,Able Sisters,...,1.0.0,Active,outdoorsy; comfy; sporty,AccessoryMouth,No,For sale,AccessoryMouthBandageSkin,4677,2qFT5iPkk8bREvpkj,0
3,beak,Yellow,No,490,122,Yellow,Yellow,1x1,NaN,Able Sisters,...,1.0.0,Cute,fairy tale; party; theatrical,AccessoryMouthInvisibleNose,No,For sale,AccessoryMouthBeakYellow,3549,T5CpsJi4xBSachNL5,0
4,birthday shades,Yellow,No,NFS,620,Yellow,Red,1x1,NaN,Birthday,...,1.0.0,Gorgeous,party,AccessoryEye,Yes,Not for sale,AccessoryGlassBirthday0,4510,S6CiB9ZvzBTMhEnDz,0
